In [1]:
import keras
import joblib

from tensorflow.keras import Sequential
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input

In [2]:
dataset = joblib.load('../../assets/pkl/update/dataset/dataset_v01.pkl')
labels = joblib.load('../../assets/pkl/update/dataset/labels_v01.pkl')

In [3]:
n_samples, n_channels, n_timesteps = dataset.shape
dataset_2d = dataset.reshape((n_samples, n_channels * n_timesteps))

In [4]:
smote = SMOTE(random_state=42)
dataset_resampled, labels_resampled = smote.fit_resample(dataset_2d, labels)

In [5]:
dataset_resampled = dataset_resampled.reshape((dataset_resampled.shape[0], n_channels, n_timesteps))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(dataset_resampled, labels_resampled, test_size=0.2, random_state=42)

In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, n_channels)).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, n_channels)).reshape(X_test.shape)

In [8]:
ohe = OneHotEncoder(sparse_output=False)
y_train = ohe.fit_transform(y_train.reshape(-1, 1))
y_test = ohe.transform(y_test.reshape(-1, 1))

In [9]:
X_train = X_train.reshape(X_train.shape[0], n_channels, n_timesteps, 1)
X_test = X_test.reshape(X_test.shape[0], n_channels, n_timesteps, 1)

In [10]:
model = Sequential([
    Input(shape=(n_channels, n_timesteps, 1)),
    Conv2D(32, (2, 2), strides=2, padding='same', activation='relu'),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(16, (2, 2), strides=2, padding='same', activation='relu'),
    MaxPooling2D((2, 2), padding='same'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(y_train.shape[1], activation='softmax')
])

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 55ms/step - accuracy: 0.4188 - loss: 1.4750 - val_accuracy: 0.7025 - val_loss: 0.9881
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.7771 - loss: 0.8338 - val_accuracy: 0.9587 - val_loss: 0.3728
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9774 - loss: 0.2909 - val_accuracy: 0.9917 - val_loss: 0.0843
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.0659 - val_accuracy: 0.9917 - val_loss: 0.0298
Epoch 5/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0153 - val_accuracy: 1.0000 - val_loss: 0.0075
Epoch 6/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0057 - val_accuracy: 1.0000 - val_loss: 0.0038
Epoch 7/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 1.0000 - loss: 0.0031 - val_accuracy: 1.0000 - val_loss: 0.0031
Epoch 8/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0020 - val_accuracy: 1.0000 - v

In [12]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9930 - loss: 0.0306 
Test accuracy: 0.9867549538612366


In [14]:
model.save("../../assets/model/update_v2/resampling/model.h5")
keras.saving.save_model(model, "../../assets/model/update_v2/resampling/model.keras")
model.save_weights("../../assets/model/update_v2/resampling/model_weight.weights.h5")

print("Save done!")

Save done!
